In [ ]:
import nltk
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize, pos_tag
import re, unicodedata
from empath import Empath
from textblob import TextBlob 
import pandas as pd
import csv


def length(string):
    if string=='':
        return [0,0,0]
    token_word=string.split()
    no_ele=sum_len=largest=0
    for ele in token_word:
        l=len(ele)
        sum_len+=l
        if l>0:                                        
            no_ele+=1
        if largest<l:
             largest=l
    return no_ele

def average_token_count(string):
    if string=='':
            return 0
    token_word=string.split()
    no_ele=sum_len=largest=0
    for ele in token_word:
        l=len(ele)
        sum_len+=l
        if l>0:                                        
            no_ele+=1
        if largest<l:
            largest=l
        
    return float(sum_len)/no_ele


def largest_token(string):
    if string=='':
        return 0
    token_word=string.split()
    no_ele=sum_len=largest=0
    for ele in token_word:
        l=len(ele)
        sum_len+=l
        if l>0:                                        
            no_ele+=1
        if largest<l:
            largest=l
    return largest


def remove_accents(string):
    nfkd = unicodedata.normalize('NFKD', string)
    return u"".join([c for c in nfkd if not unicodedata.combining(c)])





def tokenize(poem):

    tokens = []
    poem_without_eol=''.join([item.rstrip('\n') for item in poem])
     
    # Problematic characters to replace before regex
    replacements = {u'-': u' ', u'—': u' ', u'\'d': u'ed'}

    for original, replacement in replacements.items():
        replaced = poem_without_eol.replace(original, replacement)
    replaced = remove_accents(replaced)

    # Keep apostrophes, discard other non-alphanumeric symbols
    cleaned = re.sub(r'[^0-9a-zA-Z\s\']', '', replaced)

    for line in cleaned.split('\n'):
        tokens.append([word for word in line.strip().split()])

    return tokens

def empath_finder(poem):
    lexicon = Empath()
    poem_without_eol=''.join([item.rstrip('\n') for item in poem])
    return lexicon.analyze(poem_without_eol, normalize=True)

def clean_string(poem):
    poem_without_eol=''.join([item.rstrip('\n') for item in poem])
    return poem_without_eol

def get_sentiment( string): 
        
        # create TextBlob object of passed tweet text 
        analysis = TextBlob(clean_string(string)) 
        # set sentiment 
        if analysis.sentiment.polarity > 0: 
            return 'positive'
        elif analysis.sentiment.polarity == 0: 
            return 'neutral'
        else: 
            return 'negative'


def feature_extract(poem):

        Feature={}  
       
        nltk_tokens = nltk.sent_tokenize(poem)
        
        #Feature['poem']=poem

        #Feature['tokens']=tokenize(nltk_tokens)

        Feature['avg_token_length']=average_token_count(poem)
        Feature['token_count']=length(poem)
        Feature['largest_token'] = largest_token(poem)
        
        Feature['empath']=empath_finder(poem)
        
        Feature['sentiment']=get_sentiment(poem)
        
        return Feature
 







f=open( r"C:\Users\user\Desktop\RATULA\MINOR 2\feature_poem5.csv",'w+',encoding='utf-8')
csvWriter = csv.writer(f)
csvWriter.writerow(["Average_token count", "length", "largest_token","empath values","sentiment","type"])
df=pd.read_csv(r"C:/Users/user/Desktop/RATULA/MINOR 2/all.csv")

df1 = df.iloc[:,1:5]
for index,row in df1.iterrows():
    csvWriter.writerow([average_token_count(row['content']), length(row['content']), largest_token(row['content']), empath_finder(row['content']), get_sentiment(row['content']),row['type']])


